# 가장 큰 이스포츠 대회는 무엇일까?
* 출전 선수가 가장 많은 대회는 무엇일까?
* 총 상금이 많은 게임은 무엇일까?

## 라이브러리, 데이터 import
* 데이터 출처 : https://www.kaggle.com/datasets/jackdaoud/esports-earnings-for-players-teams-by-game?select=country-and-continent-codes-list.csv

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

%matplotlib inline

country= pd.read_csv('country-and-continent-codes-list.csv')
players= pd.read_csv('highest_earning_players.csv')
teams= pd.read_csv('highest_earning_teams.csv')

print(country.shape, players.shape, teams.shape)

(262, 6) (1000, 8) (928, 6)


## 데이터 확인

In [5]:
country.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [6]:
players.head()

,PlayerId,NameFirst,NameLast,CurrentHandle,CountryCode,TotalUSDPrize,Game,Genre
0,3883,Peter,Rasmussen,dupreeh,dk,1822989.41,Counter-Strike: Global Offensive,First-Person Shooter
1,3679,Andreas,Højsleth,Xyp9x,dk,1799288.57,Counter-Strike: Global Offensive,First-Person Shooter
2,3885,Nicolai,Reedtz,dev1ce,dk,1787489.88,Counter-Strike: Global Offensive,First-Person Shooter
3,3672,Lukas,Rossander,gla1ve,dk,1652350.75,Counter-Strike: Global Offensive,First-Person Shooter
4,17800,Emil,Reif,Magisk,dk,1416448.64,Counter-Strike: Global Offensive,First-Person Shooter


In [7]:
teams.head()

,TeamId,TeamName,TotalUSDPrize,TotalTournaments,Game,Genre
0,760,San Francisco Shock,3105000.0,7,Overwatch,First-Person Shooter
1,776,London Spitfire,1591136.5,13,Overwatch,First-Person Shooter
2,768,New York Excelsior,1572618.5,18,Overwatch,First-Person Shooter
3,773,Philadelphia Fusion,1186278.5,15,Overwatch,First-Person Shooter
4,766,Seoul Dynasty,1130000.0,6,Overwatch,First-Person Shooter


In [8]:
country.isna().sum()

Continent_Name                0
Continent_Code               43
Country_Name                  0
Two_Letter_Country_Code       1
Three_Letter_Country_Code     4
Country_Number                4
dtype: int64

In [9]:
players.isna().sum()

PlayerId         0
NameFirst        0
NameLast         0
CurrentHandle    0
CountryCode      0
TotalUSDPrize    0
Game             0
Genre            0
dtype: int64

In [10]:
teams.isna().sum()

TeamId              0
TeamName            0
TotalUSDPrize       0
TotalTournaments    0
Game                0
Genre               0
dtype: int64

## 1. 출전 선수 수가 가장 많은 대회는 무엇일까?

### 1. 장르에 따른 출전 국가, 팀, 선수 수

In [25]:
# 플레이어 수
player_cnt= players.groupby('Genre')['PlayerId'].nunique().reset_index().rename(columns= {'PlayerId' : 'Players'})

# 국가 수
country_cnt= players.groupby('Genre')['CountryCode'].nunique().reset_index().rename(columns= {'CountryCode' : 'Countries'})

# 팀 수
team_cnt= teams.groupby('Genre')['TeamId'].nunique().reset_index().rename(columns= {'TeamId' : 'Teams'})

In [27]:
player_cnt

,Genre,Players
0,Battle Royale,200
1,Collectible Card Game,100
2,First-Person Shooter,200
3,Multiplayer Online Battle Arena,400
4,Strategy,100


In [28]:
country_cnt

,Genre,Countries
0,Battle Royale,30
1,Collectible Card Game,26
2,First-Person Shooter,29
3,Multiplayer Online Battle Arena,36
4,Strategy,18


In [29]:
team_cnt

,Genre,Teams
0,Battle Royale,176
1,Collectible Card Game,80
2,First-Person Shooter,177
3,Multiplayer Online Battle Arena,260
4,Strategy,100


In [32]:
genre_cnt= pd.merge(left= country_cnt, right= team_cnt, on= 'Genre', how= 'left')
genre_cnt= pd.merge(left= genre_cnt, right= player_cnt, on= 'Genre', how= 'left').sort_values(by= 'Players', ascending= False)
genre_cnt

,Genre,Countries,Teams,Players
3,Multiplayer Online Battle Arena,36,260,400
0,Battle Royale,30,176,200
2,First-Person Shooter,29,177,200
1,Collectible Card Game,26,80,100
4,Strategy,18,100,100


* 출전 선수 기준으로 볼 때 MOBA 게임 대회의 규모가 압도적으로 큼
* 카드, 전략 게임 출전 선수 수는 같지만 출전 국가, 팀 수는 차이를 보임

### 2. 게임에 따른 대회 규모

In [46]:
# 선수
player_cnt= players.groupby(['Game', 'Genre'])['PlayerId'].nunique().reset_index().rename(columns= {'PlayerId' : 'Players'})
# 팀
team_cnt= teams.groupby(['Game', 'Genre'])['TeamId'].nunique().reset_index().rename(columns= {'TeamId' : 'Teams'})
# 국가
country_cnt= players.groupby(['Game', 'Genre'])['CountryCode'].nunique().reset_index().rename(columns= {'CountryCode' : 'Countries'})

In [49]:
game_cnt= pd.merge(left= country_cnt, right= team_cnt, on= 'Game', how= 'left')
game_cnt= pd.merge(left= game_cnt, right= player_cnt, on= 'Game', how= 'left').sort_values(by= 'Players', ascending= False)

game_cnt.drop(['Genre_y', 'Genre'], axis= 1, inplace= True)

game_cnt.rename(columns= {'Genre_x' : 'Genre'}, inplace= True)

# 선수 수가 모두 100명이므로 국가 수 기준으로 정렬
game_cnt= game_cnt.sort_values(by= 'Countries', ascending= False)

game_cnt

,Game,Genre,Countries,Teams,Players
2,Dota 2,Multiplayer Online Battle Arena,26,100,100
4,Hearthstone,Collectible Card Game,26,80,100
1,Counter-Strike: Global Offensive,First-Person Shooter,25,100,100
8,PUBG,Battle Royale,22,100,100
3,Fortnite,Battle Royale,21,100,100
9,Starcraft II,Strategy,18,100,100
5,Heroes of the Storm,Multiplayer Online Battle Arena,16,100,100
6,League of Legends,Multiplayer Online Battle Arena,16,100,100
7,Overwatch,First-Person Shooter,14,100,100
0,Arena of Valor,Multiplayer Online Battle Arena,7,48,100


* 앞선 결과와는 다른 결과를 보임
* MOBA 게임 도타 2, 카드 게임 하스스톤 대회 참가국이 26개로 가장 많음
* MOBA 대부분 타 장르 게임보다 참가국이 적음
* 히어로즈 오브 더 스톰과 롤 참가국이 같은 것은 의외임

In [11]:
players.head()

,PlayerId,NameFirst,NameLast,CurrentHandle,CountryCode,TotalUSDPrize,Game,Genre
0,3883,Peter,Rasmussen,dupreeh,dk,1822989.41,Counter-Strike: Global Offensive,First-Person Shooter
1,3679,Andreas,Højsleth,Xyp9x,dk,1799288.57,Counter-Strike: Global Offensive,First-Person Shooter
2,3885,Nicolai,Reedtz,dev1ce,dk,1787489.88,Counter-Strike: Global Offensive,First-Person Shooter
3,3672,Lukas,Rossander,gla1ve,dk,1652350.75,Counter-Strike: Global Offensive,First-Person Shooter
4,17800,Emil,Reif,Magisk,dk,1416448.64,Counter-Strike: Global Offensive,First-Person Shooter


In [12]:
teams.head()

,TeamId,TeamName,TotalUSDPrize,TotalTournaments,Game,Genre
0,760,San Francisco Shock,3105000.0,7,Overwatch,First-Person Shooter
1,776,London Spitfire,1591136.5,13,Overwatch,First-Person Shooter
2,768,New York Excelsior,1572618.5,18,Overwatch,First-Person Shooter
3,773,Philadelphia Fusion,1186278.5,15,Overwatch,First-Person Shooter
4,766,Seoul Dynasty,1130000.0,6,Overwatch,First-Person Shooter


In [13]:
country.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
